In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

from tensorflow.keras.preprocessing import image_dataset_from_directory

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = '0' #指定第一块GPU可用

config = tf.compat.v1.ConfigProto()

config.gpu_options.per_process_gpu_memory_fraction = 0.8 # 程序最多只能占用指定gpu50%的显存

config.gpu_options.allow_growth = True #程序按需申请内存

sess = tf.compat.v1.Session(config=config)

In [3]:
# 数据集中的训练和验证数据集
ROOT_DIR = "E:\code\kaggle\driver\imgs"
train_dir = os.path.join(ROOT_DIR, 'train')
valid_dir = os.path.join(ROOT_DIR, 'valid')

In [4]:
# 设置送入模型的图片的尺寸
IMG_SIZE = (160, 160)
# 因为我们验证集中每类只有30张图片，所以这里batch size就设置得小些，直接设置成10就好了
train_dataset = image_dataset_from_directory(train_dir, shuffle=True, batch_size=32, image_size=IMG_SIZE)
valid_dataset = image_dataset_from_directory(train_dir, shuffle=True, batch_size=10, image_size=IMG_SIZE)

Found 22124 files belonging to 10 classes.
Found 22124 files belonging to 10 classes.


In [5]:
# 默认文件名即为我们的类别名，这里打印出来看看
class_names = train_dataset.class_names
num_classes = len(class_names)
print(class_names)
print(valid_dataset.class_names)

['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']


In [6]:
# prefetch提供了pipeline的方式读取数据，更高效
# 参考： https://tensorflow.google.cn/guide/data_performance
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
valid_dataset = valid_dataset.prefetch(buffer_size=AUTOTUNE)

In [7]:
#Create the base model from the pre-trained model MobileNet V2
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
base_model.summary()

Model: "mobilenetv2_1.00_160"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 80, 80, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 80, 80, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 80, 80, 32)   0           bn_Conv1[0][0]                   
_______________________________________________________________________________

In [8]:
# 数据增强操作
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

In [9]:
# 数据进入MobileNetV2之前要先经过preprocess_input处理
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
# 全局平均池化层，参考： https://www.cnblogs.com/hutao722/p/10008581.html
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
# 输出层
prediction_layer = tf.keras.layers.Dense(num_classes)

In [10]:
# 输入
inputs = tf.keras.Input(shape=IMG_SHAPE)
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)

model = tf.keras.Model(inputs, outputs)

In [11]:
# 让MobileNetV2变得可训练
base_model.trainable = True
# 但是，前100层的参数让它冻结，我们只训练后面层的参数
fine_tune_at = 100
# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

In [12]:
# 定义损失函数
loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# 定义学习率
base_learning_rate = 0.00001
# 定义优化器
optimizer = tf.keras.optimizers.RMSprop(lr=base_learning_rate)

model.compile(loss=loss,
              optimizer = optimizer,
              metrics=['accuracy'])

In [13]:
# 开始训练
total_epochs =  50
history = model.fit(train_dataset,
                    epochs=total_epochs,
                    validation_data=valid_dataset)

Epoch 1/50
692/692 [==============================] - 152s 178ms/step - loss: 2.1413 - accuracy: 0.2457 - val_loss: 1.4150 - val_accuracy: 0.4708
Epoch 2/50
692/692 [==============================] - 70s 101ms/step - loss: 1.1011 - accuracy: 0.6310 - val_loss: 0.8036 - val_accuracy: 0.7132
Epoch 3/50
692/692 [==============================] - 260s 375ms/step - loss: 0.6502 - accuracy: 0.7862 - val_loss: 0.6284 - val_accuracy: 0.7778
Epoch 4/50
692/692 [==============================] - 63s 91ms/step - loss: 0.4408 - accuracy: 0.8566 - val_loss: 0.4476 - val_accuracy: 0.8512
Epoch 5/50
692/692 [==============================] - 64s 91ms/step - loss: 0.3308 - accuracy: 0.8919 - val_loss: 0.3022 - val_accuracy: 0.9024
Epoch 6/50
692/692 [==============================] - 64s 92ms/step - loss: 0.2571 - accuracy: 0.9193 - val_loss: 0.2720 - val_accuracy: 0.9134
Epoch 7/50
692/692 [==============================] - 64s 93ms/step - loss: 0.2144 - accuracy: 0.9303 - val_loss: 0.2536 - val_accu

In [14]:
# 在验证集里验证模型准确率
loss0, accuracy0 = model.evaluate(valid_dataset)
print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

2213/2213 [==============================] - 30s 14ms/step - loss: 0.0187 - accuracy: 0.9937
initial loss: 0.02
initial accuracy: 0.99


In [15]:
# 模型保存路径
MODEL_DIR = "./models/"
if not os.path.exists(MODEL_DIR):
  os.makedirs(MODEL_DIR)

In [16]:
# 保存模型
h5_dir = os.path.join(MODEL_DIR, 'ddd_mobilenet_v2.h5')
model.save(h5_dir)

In [17]:

# 我们将保持好的模型再导入，再在验证集里查看它的准确率
new_model = tf.keras.models.load_model(h5_dir)
for images, labels in valid_dataset.take(1):
  for i in range(10):
    img = np.expand_dims(images[i], axis=0)
    predictions = new_model.predict(img)    
    predictions = np.argmax(predictions[0])
    print("pred:", class_names[predictions], " label:" , class_names[labels[i]])

pred: c8  label: c8
pred: c3  label: c3
pred: c5  label: c5
pred: c4  label: c4
pred: c3  label: c3
pred: c5  label: c5
pred: c5  label: c5
pred: c7  label: c7
pred: c2  label: c2
pred: c3  label: c3
